In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage import io
import cv2
from itertools import chain, combinations

from pathlib import Path

PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.brain_merger import BrainMerger
from library.registration.algorithm import umeyama

/home/eddyod/programming/preprocessing-pipeline/src


In [2]:
def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(6, len(s)+1))

In [17]:
animal = 'Atlas'
brainManager = BrainStructureManager(animal)
brainManager.fixed_brain = BrainStructureManager('Allen')
brainManager.com_annotator_id = 1
brainManager.fixed_brain.com_annotator_id = 1
testing_structures = {'3N_L','3N_R','4N_L','4N_R','IC','LC_L','LC_R','PBG_L','PBG_R','SC', 'SNC_L','SNC_R','SNR_L','SNR_R'}
#testing_structures = {'3N_L','3N_R','4N_L'}
moving_coms = brainManager.get_coms(annotator_id=1)
allen_coms = brainManager.fixed_brain.get_coms(annotator_id=1)
allen_atlas_keys = sorted(allen_coms.keys() & moving_coms.keys())
allen_point_dict = {s:allen_coms[s] for s in allen_atlas_keys}
moving_point_dict = {s:moving_coms[s] for s in allen_atlas_keys}
common_keys = sorted(allen_coms.keys() & moving_coms.keys())

In [18]:
%%time
combos = list(powerset(testing_structures))

CPU times: user 1.35 ms, sys: 261 µs, total: 1.61 ms
Wall time: 1.62 ms


In [19]:
means = []
for i, combo in enumerate(combos):
    common_keys = sorted(allen_coms.keys() & moving_coms.keys() & set(combo))
    allen_points = np.array([allen_coms[p] for p in common_keys])
    moving_points = np.array([moving_coms[p] for p in common_keys])
    R, t = umeyama(moving_points.T, allen_points.T)
    distances = []
    for structure in allen_atlas_keys:
        (x,y,z) = allen_point_dict[structure]
        allen_point = np.array([x,y,z])  
        moving_point = np.array(moving_point_dict[structure])
        reg_point = brain_to_atlas_transform(moving_point, R, t)
        d = calculate_distance(allen_point, reg_point)
        distances.append(d)

    #print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
    means.append(np.mean(distances))
print(f'min of means = {min(means)}')



reflection detected
reflection detected
reflection detected
reflection detected
min of means = 363.26613162078104


In [14]:
allen_points = np.array([allen_coms[p] for p in testing_structures])
moving_points = np.array([moving_coms[p] for p in testing_structures])
allen_points.shape, moving_points.shape

((12, 3), (12, 3))

In [15]:
R, t = umeyama(moving_points.T, allen_points.T)
t

array([[-2122.24353486],
       [-1507.35522247],
       [  -31.72962935]])

In [16]:
distances = []
sortme = {}
diffs = {}
diff_dict = {0:'X', 1:'Y',2:'Z'}
for structure in allen_atlas_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])  
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, R, t)
    diff = np.round(abs(allen_point - reg_point),2)
    mx = np.argmax(diff)
    diffs[structure] = (diff, diff_dict[mx])
    d = calculate_distance(allen_point, reg_point)
    distances.append(d)
    sortme[structure] = d

print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'{structure} {round(d,2)} {diffs[structure]}')

n=37, min=94.96455033162336 max=1159.139634181129, mean=506.3180044233738
LC_R 94.96 (array([68.61, 41.88, 50.57]), 'X')
3N_L 126.6 (array([ 43.22, 114.32,  33.01]), 'Y')
4N_L 135.4 (array([ 22.11, 115.38,  67.31]), 'Y')
4N_R 154.23 (array([  9.82, 153.89,   3.03]), 'Y')
3N_R 156.94 (array([ 17.12, 155.88,   6.13]), 'Y')
LC_L 169.29 (array([ 44.43,  89.62, 136.58]), 'Z')
SNR_R 207.46 (array([ 77.76, 179.48,  69.12]), 'Y')
PBG_R 221.81 (array([119.44, 159.94,  96.7 ]), 'Y')
IC 222.64 (array([ 60.75, 195.84,  86.76]), 'Y')
SNR_L 249.26 (array([148.43, 200.03,   9.26]), 'Y')
5N_L 286.17 (array([191.88, 193.86,  86.57]), 'Y')
PBG_L 290.71 (array([120.81, 237.04, 117.18]), 'Y')
5N_R 326.79 (array([204.02, 254.98,  12.36]), 'Y')
SNC_L 329.43 (array([ 87.49, 241.33, 206.47]), 'Y')
7n_R 336.46 (array([265.73, 152.19, 139.38]), 'X')
7n_L 355.54 (array([234.26,  95.49, 249.82]), 'Z')
SNC_R 360.22 (array([176.87, 171.15, 263.03]), 'Z')
6N_R 368.72 (array([344.3 ,  99.55,  86.61]), 'X')
6N_L 369.5

In [ ]:
brainMerger = BrainMerger()
brainMerger.save_brain_area_data()

In [ ]:
plotpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/Atlas/plots'
csvpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/Atlas/csv'
atlaspath = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'

In [ ]:
df_allen = pd.read_csv(os.path.join(csvpath, 'Allen.csv'))
df_atlas = pd.read_csv(os.path.join(csvpath, 'Atlas.csv'))
df_atlas.head()

In [ ]:
allen_path = os.path.join(atlaspath, 'Allen_25um_sagittal.tif')
allen_img = io.imread(allen_path)
allen_img.dtype, allen_img.shape
#sagittal = np.swapaxes(allen_img,0,2)
sagittal = allen_img
print(sagittal.shape)

In [ ]:
distances = []
sortme = {}

for (i1,row1), (i2,row2) in zip(df_allen.iterrows(), df_atlas.iterrows() ): 
    structure = row1['Structure']

    fig = plt.figure()
    plt.rcParams["figure.figsize"] = [12, 8]
    x1 = row1['X']
    y1 = row1['Y']
    z1 = row1['Z']
    x2 = row2['X']
    y2 = row2['Y']
    z2 = row2['Z']
    section = int(round(z1/25, 2))
    xdistance = abs(x1 - x2)
    ydistance = abs(y1 - y2)
    zdistance = abs(z1 - z2)
    distance = calculate_distance(np.array([x1,y1,z1]), np.array([x2,y2,z2]))
    slice = sagittal[section,:,:]
    title = f'{structure} at section={section}\n'
    title += f'x distance={round(xdistance,2)} um\n'
    title += f'y distance={round(ydistance,2)} um\n'
    title += f'z distance={round(zdistance,2)} um\n'
    title += f'  distance={round(distance,2)} um'
    plt.title(title)
    ax = plt.gca()
    #ax.set_xlim([300, 600])
    ax.set_ylim(300,0)
    plt.scatter(x1/25, y1/25, s=40, marker='^', c='b', label=str('Allen ' + structure))
    plt.scatter(x2/25, y2/25, s=40, marker='o', c='g', label=str('Atlas ' + structure))
    plt.imshow(slice, cmap='gray')
    plt.legend()
    plt.close()
    #outpath = os.path.join(plotpath, 'eps', f'{structure}.eps')
    #fig.savefig(outpath, bbox_inches="tight", format='eps')
    outpath = os.path.join(plotpath, f'{structure}.png')
    fig.savefig(outpath, bbox_inches="tight")
    
    distances.append(distance)
    sortme[structure] = distance


In [ ]:
width = 0.25  # the width of the bars
multiplier = 0
x = np.arange(1) 
x + width

In [ ]:
errors = ['X error', 'Y error', 'Z error', 'Distance error']
x = np.arange(len(errors))  # the label locations
width = 0.05  # the width of the bars
for (i1,row1), (i2,row2) in zip(df_midbrain_allen.iterrows(), df_midbrain_atlas.iterrows() ): 
    structure = row1['Structure']
    #fig = plt.figure()
    fig, ax = plt.subplots(figsize=(6, 4))
    #plt.rcParams["figure.figsize"] = [10, 6]

    x1 = row1['X']
    y1 = row1['Y']
    z1 = row1['Z']
    x2 = row2['X']
    y2 = row2['Y']
    z2 = row2['Z']
    xdistance = abs(x1 - x2)
    ydistance = abs(y1 - y2)
    zdistance = abs(z1 - z2)
    distance = calculate_distance(np.array([x1,y1,z1]), np.array([x2,y2,z2]))

    counts = [xdistance, ydistance, zdistance, distance]
    bar_labels = ['X error', 'Y error', 'Z error', 'Distance error']
    bar_colors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange']
    #ax.bar(errors, counts, label=bar_labels, color=bar_colors)
    ax.set_ylabel('Error in um')
    # Label with specially formatted floats
    rects1 = ax.bar(x - width/2, counts, label=bar_labels, color=bar_colors)
    ax.bar_label(rects1, padding=2)
    ax.set_xticks(x, errors)
    ax.set_title('Errors for ' + structure)
    ax.set_ylim([0,1200])
    #ax.legend(title='Error color')
    plt.close()
    outpath = os.path.join(distancepath, 'eps', f'{structure}_error.eps')
    fig.savefig(outpath, bbox_inches="tight", format='eps')
    outpath = os.path.join(distancepath, f'{structure}_error.png')
    fig.savefig(outpath, bbox_inches="tight")


In [ ]:
print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    
    #print(f'{structure} distance from Allen={round(d,2)} micrometers')
    print('<tr>')
    print(f'<td><img src="https://www.brainsharer.org/images/{structure}.png"></td>')
    print(f'<td><img src="https://www.brainsharer.org/images/{structure}_error.png"></td>')
    print('</tr>')


In [ ]:
midbrain_distances = df_midbrain_distance.distance
all_distances = df_all_distance.distance
brainstem_distances = df_brainstem_distance.distance
fig, ax = plt.subplots(figsize=(8, 6))
columns = [midbrain_distances, all_distances, brainstem_distances]
ax.boxplot(columns, patch_artist=True)
ax.set_title("Distances between the DK atlas and the Allen (um)")
title1 = f"Using mid-brain\nn={len(midbrain_distances)}, min={round(min(midbrain_distances))}"
title1 += f"\nmax={round(max(midbrain_distances))}, mean={round(np.mean(midbrain_distances))}"
title2 = f"Using all structures\nn={len(all_distances)}, min={round(min(all_distances))}"
title2 += f"\nmax={round(max(all_distances))}, mean={round(np.mean(all_distances))}"
title3 = f"Using brainstem structures\nn={len(brainstem_distances)}, min={round(min(brainstem_distances))}"
title3 += f"\nmax={round(max(brainstem_distances))}, mean={round(np.mean(brainstem_distances))}"
plt.xticks([1,2,3], [title1, title2, title3])
plt.show()

In [ ]:
len(all_distances), min(all_distances), max(all_distances), np.mean(all_distances)

In [ ]:
for i, row in df_midbrain_distance.iterrows():
    print(f'{row[0]} distance={round(row[1],2)}')

In [ ]:
for i, row in df_brainstem_distance.iterrows():
    print(f'<tr><td>{row[0]}</td> <td>{round(row[1],2)}</td></tr>')

In [ ]:
xmin = 8000
xmax = 13000
ymin = 2000
ymax = 6000
zmin = 0
zmax = 25*456

In [ ]:
df_midbrain_atlas.head(2)

In [ ]:
for color, label in zip('bg', ['midbrain', 'brainstem']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.area == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.area == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
    #ax.annotate('x', (atlas_subset.X, atlas_subset.Y))
    #ax.annotate('x', xy=(atlas_subset.X, atlas_subset.Y))
plt.title('Mid-brain Allen/Altas X Y scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
for k, v in df_midbrain_atlas.iterrows():
    ax.annotate(v['Structure'],
                xy=(v['X'], v['Y']),
                xytext=(10, 10), textcoords='offset points',
                family='sans-serif', fontsize=10, color='black')

In [ ]:
for color, label in zip('bgr', ['midbrain', 'brainstem']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.area == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.area == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Mid-brain Allen/Altas X Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)